## Librery & Data 

In [3]:
# Importamos la librerias 
import pandas as pd 
import ast
import sklearn 

In [4]:
# traemos la data 
games = pd.read_parquet("D:\Documentos\Courses\Henry\Henry Students\Labs\Cohorte 07\PI_1\PI MLOps - STEAM\Data\parquet\Datos Generales\steam_games.parquet")
items = pd.read_parquet("D:\Documentos\Courses\Henry\Henry Students\Labs\Cohorte 07\PI_1\PI MLOps - STEAM\Data\parquet\\Datos Generales\items.parquet")
reviews =pd.read_parquet("D:\\Documentos\\Courses\\Henry\\Henry Students\\Labs\\Cohorte 07\\PI_1\\PI MLOps - STEAM\\Data\\parquet\\Datos Generales\\reviews.parquet")

In [5]:
# Tener un punto de reinicio
df_games  = pd.DataFrame(games)
df_items = pd.DataFrame(items)
df_reviews = pd.DataFrame(reviews)

#### Primera funcion
def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
<div>Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}</div>

In [6]:
# Veamos info del Dataframe que vamos a usar 
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 88310 to 120444
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     32132 non-null  object 
 1   name       32132 non-null  object 
 2   specs      31463 non-null  object 
 3   price      32132 non-null  float64
 4   item_id    32132 non-null  int32  
 5   developer  32132 non-null  object 
 6   Year       32132 non-null  Int64  
dtypes: Int64(1), float64(1), int32(1), object(4)
memory usage: 1.9+ MB


In [7]:
# # Veamos info del Dataframe que vamos a usar 
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285249 entries, 0 to 5170013
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   user_url          object
 2   item_id           int32 
 3   item_name         object
 4   playtime_forever  int32 
 5   playtime_2weeks   int32 
dtypes: int32(3), object(3)
memory usage: 137.9+ MB


In [8]:
# Observamos la columna "Item_id"
df_items['item_id']

0              10
2              30
8             300
9             240
10           3830
            ...  
5170007    304930
5170008    227940
5170011    388490
5170012    521570
5170013    519140
Name: item_id, Length: 3285249, dtype: int32

In [9]:
# # Observamos la columna "genres"
df_games["genres"]

88310     ['Action', 'Casual', 'Indie', 'Simulation', 'S...
88311          ['Free to Play', 'Indie', 'RPG', 'Strategy']
88312     ['Casual', 'Free to Play', 'Indie', 'Simulatio...
88313                     ['Action', 'Adventure', 'Casual']
88314               ['Action', 'Indie', 'Casual', 'Sports']
                                ...                        
120440        ['Casual', 'Indie', 'Simulation', 'Strategy']
120441                      ['Casual', 'Indie', 'Strategy']
120442                    ['Indie', 'Racing', 'Simulation']
120443                                  ['Casual', 'Indie']
120444    ['Early Access', 'Adventure', 'Indie', 'Action...
Name: genres, Length: 32132, dtype: object

In [10]:
# Hacemos un merge en base con la columna item_id
df_games_items = pd.merge(df_games, df_items, on='item_id')

In [11]:
# Borraremos columnas no deseadas
df_games_items = df_games_items.drop(columns= ["item_name", "user_url", "user_id", "developer", "specs", "price", "name"])
df_games_items.columns

Index(['genres', 'item_id', 'Year', 'playtime_forever', 'playtime_2weeks'], dtype='object')

In [12]:
# Convertir las cadenas que representan listas a listas reales
df_games_items['genres'] = df_games_items['genres'].apply(ast.literal_eval)
# Los datos de la columna estan en una lista, queremos esten por filas
# Usaremos el metodo explode
df_games_items = df_games_items.explode("genres", ignore_index=False)
df_games_items

,genres,item_id,Year,playtime_forever,playtime_2weeks
0,Action,282010,1997,5,0
0,Indie,282010,1997,5,0
0,Racing,282010,1997,5,0
1,Action,282010,1997,13,0
1,Indie,282010,1997,13,0
...,...,...,...,...,...
2868053,Action,80,2004,203,0
2868054,Action,80,2004,244,0
2868055,Action,80,2004,7,0
2868056,Action,80,2004,9,0


In [13]:
# Solo queremos devolver el tiempo que más se juega, no menos
df_games_items = df_games_items[~(df_games_items[['playtime_forever', 'playtime_2weeks']] == 0).any(axis=1)]
df_games_items.shape[0]

386767

In [14]:
# Calcular la mediana de la columna 'playtime_forever'
promedio_playtime = df_games_items['playtime_forever'].median()

# Imprimir el promedio
print("La mediana de playtime_forever es:", promedio_playtime)


La mediana de playtime_forever es: 1747.0


In [15]:
# Eliminaremos los valores de la mediana de la columna playitme_forever
df_games_items = df_games_items[~(df_games_items[['playtime_forever', 'playtime_2weeks']] <= 500).any(axis=1)]
df_games_items.shape[0]

66718

In [16]:
df_games_items = df_games_items.reset_index()

In [17]:
df_games_items.head()

,index,genres,item_id,Year,playtime_forever,playtime_2weeks
0,1277,Action,70,1998,836,815
1,1623,Action,70,1998,2415,602
2,2496,Action,70,1998,1205,535
3,5899,Action,2400,2006,1574,1574
4,5899,Indie,2400,2006,1574,1574


In [18]:
df_games_items = df_games_items.drop(columns= ["index"])

In [19]:
df_games_items['playtime_forever'].isna().sum()

0

##### Ya tenemos los datos listos creamos la funcion 

In [22]:
def PlaytimeGenre(genero):
    filtro = df_games_items[df_games_items['genres'] == genero]
    group = filtro[filtro['playtime_forever'] == filtro['playtime_forever'].max()]
    max_year = group['Year'].values[0]
    return {"Año de lanzamiento con más horas jugadas para Género " + genero: max_year}

In [24]:
PlaytimeGenre("Action")

{'Año de lanzamiento con más horas jugadas para Género Action': 2004}

In [39]:
# Parquet
df_games_items.to_parquet("Playtime.parquet")

In [40]:
# Json
df_games_items.to_json("Playtime1.json")

#### Segunda funcion
def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
<div> Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
</div>